In [1]:
# 기본 라이브러리 임포트
import pandas as pd             
import numpy as np             
import matplotlib.pyplot as plt 
import seaborn as sns           
import gc                       # 가비지 컬렉션(메모리 해제)
import re                       # 정규 표현식 처리
from collections import defaultdict  # 기본값이 있는 딕셔너리 생성

# 경고 메시지 억제
import warnings
warnings.filterwarnings('ignore')   

# 그래프 스타일 설정
sns.set()                           # seaborn 기본 스타일 적용

# matplotlib 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 폰트 설정
# plt.rcParams['font.family'] = 'AppleGothic'  
plt.rcParams['figure.figsize'] = (12, 6)       # 그림 크기 설정 (가로, 세로)
plt.rcParams['font.size'] = 14                 # 폰트 크기 설정
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 기호 깨짐 방지

# 결측치 시각화 라이브러리 임포트
import missingno                          # 결측치 분포를 시각화하는 유틸리티

# 범주형 변수 레이블 인코딩을 위한 도구 임포트
from sklearn.preprocessing import LabelEncoder

# 중복 조합 생성에 사용할 product 함수 임포트
from itertools import product

# 회귀 및 통계 분석을 위한 statsmodels 임포트
import statsmodels.api as sm

# 다중공선성 진단용 VIF 계산 함수 임포트
from statsmodels.stats.outliers_influence import variance_inflation_factor

### 파일 병합 및 VIF 값 확인 후 다중공선성 유발 컬럼 제거

In [2]:
# VIF > 10 칼럼 제거 함수 정의
def remove_high_vif(df, id_col='ID', thresh=10):
    cols = [c for c in df.select_dtypes(include='number').columns if c != id_col]
    removed = []
    while True:
        X = df[cols].fillna(0)
        X_const = sm.add_constant(X)
        vif_vals = [
            variance_inflation_factor(X_const.values, i+1)
            for i in range(len(cols))
        ]
        vif_df = pd.DataFrame({'feature': cols, 'vif': vif_vals}) \
                    .sort_values('vif', ascending=False)
        high = vif_df.loc[vif_df['vif'] > thresh, 'feature']
        if high.empty:
            break
        to_remove = high.iloc[0]
        removed.append(to_remove)
        cols.remove(to_remove)
    return df.drop(columns=removed)

In [3]:
# 마케팅정보 가져오기
marketing_df   = pd.read_csv('마케팅정보_최종.csv')

# 성과정보 가져오기 후 VIF 필터링
performance_df = remove_high_vif(pd.read_csv('성과정보12월_feat_select.csv'))

# 신용정보 가져오기 후 VIF 필터링
credit_df      = remove_high_vif(pd.read_csv('신용정보12월_feat_select.csv'))

# 채널정보 가져오기 후 VIF 필터링
channel_df     = remove_high_vif(pd.read_csv('채널정보_최종.csv'))

# 회원정보 가져오기 후 VIF 필터링
member_df      = remove_high_vif(pd.read_csv('회원정보12월_feat_select.csv'))

# 잔액정보 가져오기 후 VIF 필터링
balance_df     = remove_high_vif(pd.read_csv('charge_balance_201812.csv'))

# 승인매출정보 가져오기 후 VIF 필터링
approval_df    = remove_high_vif(pd.read_csv('승인매출정보_최종.csv'))

In [4]:
# 보조 데이터프레임에서 중복된 Segment 제거
dfs = [performance_df, credit_df, channel_df, member_df, balance_df, approval_df]  
for df in dfs:
    if 'Segment' in df.columns:
        df.drop(columns=['Segment'], inplace=True)

# 병합 기준 df 복사
merged_df = marketing_df.copy()

# 순차 병합하면서 ID 중복 바로 제거
for df in dfs:
    merged_df = (
        merged_df
        .merge(df, on='ID', how='left')
        .drop_duplicates(subset='ID')
    )

# 기준년월 제거
merged_df.drop(columns=['기준년월'], inplace=True)

In [5]:
# TRAIN/Test 데이터 분리
train_df = merged_df[merged_df['ID'].str.startswith('TRAIN')]
test_df  = merged_df[merged_df['ID'].str.startswith('TEST')]

# 분리 결과 확인
display(train_df.head())
display(test_df.head())

,ID,Segment,컨택건수_이용유도_청구서_B0M,컨택건수_이용유도_TM_R6M,컨택건수_이용유도_EM_R6M,컨택건수_이용유도_EM_B0M,컨택건수_이용유도_인터넷_R6M,컨택건수_이용유도_LMS_R6M,컨택건수_이용유도_LMS_B0M,캠페인접촉일수_R12M_num,...,이용금액대_ord,정상청구원금_B2M,연속유실적개월수_기본_24M_카드,최대이용금액_체크_R12M,최대이용금액_CA_R12M,이용금액_오프라인_B0M,이용금액_체크_R12M,이용개월수_일시불_R6M,이용건수_신용_R12M,이용금액_일시불_R12M
0,TRAIN_000000,D,0,0,78,13,0,12,2,1,...,6,15251,17,998,12264,3931,7824,6,147,24782
1,TRAIN_000001,E,0,0,0,0,3,4,2,15,...,5,2776,17,0,3516,4033,-414,6,177,53959
2,TRAIN_000002,C,0,0,8,1,0,0,0,1,...,6,23325,8,0,69186,10536,-414,6,149,60220
3,TRAIN_000003,D,0,0,20,2,0,12,2,1,...,6,18808,24,0,9802,3940,-414,6,107,16649
4,TRAIN_000004,E,1,3,0,0,0,14,2,1,...,2,0,0,3910,0,0,12988,1,-1,-861


,ID,Segment,컨택건수_이용유도_청구서_B0M,컨택건수_이용유도_TM_R6M,컨택건수_이용유도_EM_R6M,컨택건수_이용유도_EM_B0M,컨택건수_이용유도_인터넷_R6M,컨택건수_이용유도_LMS_R6M,컨택건수_이용유도_LMS_B0M,캠페인접촉일수_R12M_num,...,이용금액대_ord,정상청구원금_B2M,연속유실적개월수_기본_24M_카드,최대이용금액_체크_R12M,최대이용금액_CA_R12M,이용금액_오프라인_B0M,이용금액_체크_R12M,이용개월수_일시불_R6M,이용건수_신용_R12M,이용금액_일시불_R12M
400000,TEST_00000,NaN,0,0,6,1,2,12,2,1,...,3,4884,8,0,0,9741,1890,6,110,62221
400001,TEST_00001,NaN,0,0,6,2,0,0,0,5,...,3,10652,8,6832,8768,3490,47520,6,54,7081
400002,TEST_00002,NaN,0,0,5,2,2,12,2,15,...,3,10796,24,0,0,10759,-414,6,525,71644
400003,TEST_00003,NaN,0,0,11,1,3,0,0,15,...,4,1266,9,0,0,965,144,6,317,8418
400004,TEST_00004,NaN,0,0,0,0,0,12,2,1,...,4,1563,17,2740,0,985,6664,6,182,15258


In [8]:
# 숫자형 칼럼 중 ID, Segment 제외해 리스트 생성
numeric_cols = [
    c for c in train_df.select_dtypes(include='number').columns
    if c not in ['ID', 'Segment']
]

# 제거된 칼럼 기록용 리스트 초기화
removed = []

# VIF 반복 계산·제거
while True:
    # 설명 변수 설정
    X = train_df[numeric_cols]
    # 상수항 추가
    X_const = sm.add_constant(X)
    # VIF 계산해 DataFrame 생성
    vif_data = pd.DataFrame({
        'feature': X_const.columns,
        'vif': [
            variance_inflation_factor(X_const.values, i)
            for i in range(X_const.shape[1])
        ]
    })
    # 상수항 제외하고 정렬
    vif_data = (
        vif_data[vif_data['feature'] != 'const']
        .sort_values('vif', ascending=False)
        .reset_index(drop=True)
    )
    
    # 최고 VIF와 해당 칼럼 확인
    max_vif = vif_data.loc[0, 'vif']
    top_feature = vif_data.loc[0, 'feature']
    
    # 최고 VIF가 10 이하이면 종료
    if max_vif <= 10:
        break
    
    # 최고 VIF 칼럼 제거
    removed.append(top_feature)
    train_df.drop(columns=[top_feature], inplace=True)
    numeric_cols.remove(top_feature)

# 최종 VIF 결과 출력
display(vif_data)
print('제거된 칼럼:', removed)

,feature,vif
0,이용여부_3M_해외겸용_신용_본인,9.539121
1,유효카드수_신용,9.372655
2,증감율_이용금액_일시불_전월,9.130029
3,증감율_이용금액_신용_전월,9.009151
4,소지카드수_유효_신용,8.975804
...,...,...
101,IB문의건수_CS_R6M,1.076137
102,컨택건수_CA_청구서_R6M,1.023338
103,자발한도감액금액_R12M,1.009011
104,컨택건수_리볼빙_LMS_R6M,1.001891


제거된 칼럼: ['최대이용금액_체크_R12M', '포인트_적립포인트_R3M', '잔액_신판ca최대한도소진율_r6m', '_1순위카드이용금액', '캠페인접촉건수_R12M_num', '정상청구원금_B2M', '회원여부_이용가능_카드론', '컨택건수_이용유도_EM_R6M']


In [9]:
# train_df 저장
train_df.to_csv('final_train.csv', index=False, encoding='utf-8-sig')

# test_df 저장
test_df.to_csv('final_test.csv', index=False, encoding='utf-8-sig')